
# Hand Tracking Model Conversion to TFLite

Last Updated: 09/09/2021

> [**Open in Google Colab**](https://colab.research.google.com/github/shubham0204/Google_Colab_Notebooks/blob/main/Hand_Tracking_Model_TFLite_Conversion.ipynb)



## 1) Installing TensorFlow 1.x

As we know that Colab is shipped with TensorFlow 2.x by default ( you may check this if you need ), we need to install TensorFlow 1.15.0, the last version of TensorFlow 1.x family.

This is important because the project [victordibia/handtracking](https://github.com/victordibia/handtracking) seems to be created with TensorFlow 1.x.


In [ ]:

!pip install tensorflow==1.15.0



## 2) Cloning TF Object Detection API

In order to convert the checkpoint files to the TFLite format, we'll need the [TensorFlow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection), as it contains various scripts to perform the conversion.


There's no separate API, its just a part of the [tensorflow/models](https://github.com/tensorflow/models) repository.


In [ ]:

!git clone --depth 1 https://github.com/tensorflow/models



We also clone Victor Dibia's [handtracking](https://github.com/victordibia/handtracking) repository in order to use the trained checkpoints.


In [ ]:

!git clone https://github.com/victordibia/handtracking



## 3) Installing TF Object Detection API

Once we have cloned the TF OD API, we're ready to install the packages required by the API. This is easy to do in Colab, as we only need to run some commands, which can also be seen here.

You can go through the installation steps [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html).


In [ ]:

%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf1/setup.py .
python -m pip install .



## 4) Exporting the trained checkpoints to a TFLite-compatible graph

Using the `export_tflite_ssd_graph.py` script in the `models/research/object_detection` directory, we'll convert the trained checkpoints, present in `handtracking/model-checkpoint` directory, to a TFLite-compatible graph.

The script takes in three arguments,

1. `pipeline_config_path` : This is the path to the `.config` file of your trained model.

2. `trained_checkpoint_prefix` : Prefix of trained checkpoints we need to convert .

3. `max_detections` : Number of detections the model will make. **This is important**, as it determines the output shapes of your model. We'll come to this in step 6.

The outputs of this script are two files, `tflite_graph.pb` and `tflite_graph.pbtxt` in the `outputs` directory.



In [ ]:

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path /content/handtracking/model-checkpoint/ssdlitemobilenetv2/data_ssdlite.config \
    --trained_checkpoint_prefix /content/handtracking/model-checkpoint/ssdlitemobilenetv2/out_model.ckpt-19040 \
    --output_directory outputs\
    --max_detections 10



## 5) Convert the TFLite-compatible graph to a TFLite model ( .tflite )

Using the utility tool `tflite_convert`, we convert the TFLite-compatible graph to a TFLite ( .tflite ) model. 




In [ ]:

!tflite_convert \
--graph_def_file=/content/outputs/tflite_graph.pb \
--output_file=/content/outputs/model.tflite \
--output_format=TFLITE \
--input_arrays=normalized_input_image_tensor \
--input_shapes=1,300,300,3 \
--inference_type=FLOAT \
--output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
--allow_custom_ops




## 6) Inspect the input/output shapes of the TFLite model

Using the `tf.lite.Interpreter` class, we'll check the input and output shapes ( and dtypes ) of our model.


In [ ]:

import tensorflow as tf
import pprint

interpreter = tf.lite.Interpreter( '/content/outputs/model.tflite' )
interpreter.allocate_tensors()

pprint.pprint( interpreter.get_input_details())
pprint.pprint( interpreter.get_output_details() )
